In [1]:
import pandas as pd
from googletrans import Translator
import re
from collections import defaultdict
import time
import numpy as np
import enchant
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence

In [32]:
def split_into_lines(songtext):
    lines = songtext.split('\\n')
    return lines    

In [33]:
def get_line_by_line_language(songtext):
    languages= defaultdict()
    lines = songtext.split('\\n')
    for i, line in enumerate(lines):
        result= translator.detect(line)
        result = str(result)
        lang_line= result.split('=')[1].split(',')[0]
        languages[i] = lang_line
    print(languages[0])
    
    return languages    


In [41]:
def convert (string):
    return eval(string)

In [44]:
def find_english_lines(dictionary):
    english_list = []
    for k,v in dictionary.items():
        if v== 'en':
            english_list.append(k)
    return english_list


In [45]:
def return_english_lines(mixed_lines, english_list):
    english_lines = [mixed_lines[i] for i in english_list]
    return english_lines

In [72]:
def tokenize_corpus(corpus):
    songs_tokenized = []
    for song in corpus: 
        song_tokens = text_to_word_sequence(song, filters='!"#$%&()*+,.:;=?@[\\]^_`{|}~\t\n')
        songs_tokenized.append(song_tokens)
    return songs_tokenized

In [59]:
def clean_up_songs(tokenized_songs):
    eng_songs = []
    for song in tokenized_songs:
        eng_words = []
        for word in song:
            if d.check(word):
                eng_words.append(word)
        eng_songs.append(eng_words)
    return eng_songs

In [68]:
def generate_train_dataset(tokenized_corpus, window=10):
    train_data = []
    train_label = []
    for song in tokenized_corpus:
        len_song = len(song)        
        for i in range(0,len_song-window+1):
            train_data.append(song[i:i+window-1])
            train_label.append(song[i+window-1])
    return np.array(train_data), np.array(train_label)

Make a new column for lyrics of each song split line by line, and another containing a dictionary determining the language of each line using Google's Translator

In [31]:
df = pd.read_csv('contestants.csv')

In [34]:
translator = Translator()

In [35]:
df['split_lyrics'] = df['lyrics'].apply(split_into_lines)

In [36]:
#This step takes a lot of computational time
df['line_languages'] = df['lyrics'].apply(get_line_by_line_language)

fr


KeyboardInterrupt: 

Save/load dictionary as a csv file

In [38]:
#Save/load resulting dataframe as a csv
#df.to_csv('eurovision_full.csv')
df = pd.read_csv('eurovision_full.csv')

In [39]:
df = df[['lyrics', 'line_languages']]

In [40]:
df['split_lyrics'] = df['lyrics'].apply(split_into_lines)

In [42]:
#Defaultdict stored as a string, must be converted back into dict
df['line_languages'] = df['line_languages'].apply(convert)

In [43]:
df = df[['split_lyrics', 'line_languages']]

In [48]:
english_lyrics=[]
for i, row in df.iterrows():
    english_lines = find_english_lines(row['line_languages'])
    mixedlines = row['split_lyrics']
    row_english = return_english_lines(mixedlines, english_lines)
    english_lyrics.append(row_english)

In [51]:
english_corpus =[' '.join(i) for i in english_lyrics]

In [53]:
d = enchant.Dict("en_US")

In [73]:
tokenized_songs = tokenize_corpus(english_corpus)

In [83]:
corpus=clean_up_songs(tokenized_songs)

In [279]:
# Pickles the corpus
with open('./corpus', 'wb') as fp:
    pickle.dump(corpus, fp) 

In [84]:
word_tokenizer = Tokenizer(filters='!"#$%&()*+,.:;=?@[\\]^_`{|}~\t\n')
word_tokenizer.fit_on_texts(corpus) # need to flatten the array

In [287]:
# Pickles the word tokenizer object
with open('./tokenizer', 'wb') as fp:
    pickle.dump(word_tokenizer, fp) 